In [1]:
import numpy as np
from src import config
from src import MyUtil as myUtil
print('done import')

done import


In [3]:

recordNames = config.NAME_OF_RECORD
numRecord = len(recordNames)
numTrain = config.NUMBER_OF_TRAIN_RECORD
trainRqa = []
trainLabel = []

# for iRecord in range(0, numTrain):
for iRecord in range(10, numRecord):
    rqa, label, _ = myUtil.loadRqa(recordNames[iRecord], 'train')
    trainRqa = np.append(trainRqa, rqa, axis=0) if len(trainRqa) > 0 else rqa
    trainLabel = np.append(trainLabel, label, axis=0) if len(trainLabel) > 0 else label

testRqa = []
testLabel = []
for iRecord in range(0, 10):
    rqa, label, _ = myUtil.loadRqa(recordNames[iRecord], 'test')
    testRqa = np.append(testRqa, rqa, axis=0) if len(testRqa) > 0 else rqa
    testLabel = np.append(testLabel, label, axis=0) if len(testLabel) > 0 else label
print('done load data')


done load data


In [4]:
print('trainRqa: ', trainRqa.shape)
# print(trainRqa[-10:])
print('trainLabel: ', trainLabel.shape)
print('testRqa: ', testRqa.shape)
# print(testRqa[-10:])
print('testLabel: ', testLabel.shape)

unique, count = np.unique(trainLabel, return_counts=True)
print('trainLabel unique: ', dict(zip(unique, count)))
unique, count = np.unique(testLabel, return_counts=True)
print('testLabel unique: ', dict(zip(unique, count)))

trainRqa:  (15443, 9)
trainLabel:  (15443,)
testRqa:  (4407, 9)
testLabel:  (4407,)
trainLabel unique:  {0: 11013, 1: 4430}
testLabel unique:  {0: 1722, 1: 2685}


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
# X, y = make_classification(n_samples=len(trainRqa), n_features=2,
#                            n_informative=2, n_redundant=0,
#                            random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=6, n_estimators=100)
# clf = RandomForestClassifier(max_depth=6, n_estimators=100)
# clf = RandomForestClassifier()
clf.fit(trainRqa, trainLabel)

print('done fit')

done fit


In [25]:
from hpsklearn import HyperoptEstimator, extra_trees
from hyperopt import tpe

estim = HyperoptEstimator(classifier=clf,
                          algo=tpe.suggest,
                          max_evals=10,
                          trial_timeout=300)
estim.fit(trainRqa, trainLabel)
print(estim.best_model())


print('param: ', clf.get_params())

100%|██████████| 10/10 [00:03<00:00,  2.75trial/s, best loss: 0.16574943347361604]
{'learner': RandomForestClassifier(), 'preprocs': (StandardScaler(),), 'ex_preprocs': ()}
param:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [7]:

from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

pre = clf.predict(testRqa)
print('pre: ', pre)
test = accuracy_score(testLabel, pre)
print(test)

print('recall: ', recall_score(testLabel, pre))
print('percision: ', precision_score(testLabel, pre))
print('confusion_matrix: \n', confusion_matrix(testLabel, pre))
print('done Test')


pre:  [1 1 1 ... 0 0 0]
0.7227138643067846
recall:  0.5929236499068902
percision:  0.9250435793143521
confusion_matrix: 
 [[1593  129]
 [1093 1592]]
done Test


100%|██████████| 10/10 [00:02<00:00,  4.16trial/s, best loss: 0.1544189057947556]


{'learner': RandomForestClassifier(max_depth=6), 'preprocs': (Normalizer(),), 'ex_preprocs': ()}


In [11]:
print(estim.score(testRqa, testLabel))
# 0.962785714286

print(estim.best_model())
clf1 = estim.best_model()
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

pre = clf1.predict(testRqa)
print('pre: ', pre)
test = accuracy_score(testLabel, pre)
print(test)

print('recall: ', recall_score(testLabel, pre))
print('percision: ', precision_score(testLabel, pre))
print('confusion_matrix: \n', confusion_matrix(testLabel, pre))
print('done Test')
print('done fit')



0.7258906285454958
{'learner': RandomForestClassifier(max_depth=6), 'preprocs': (), 'ex_preprocs': ()}


AttributeError: 'dict' object has no attribute 'predict'